# Vulnerable Spring

[vuln-spring](https://github.com/HooliCorp/vuln-spring) is an intentionally vulnerable spring application to test SAST tools. The purpose of this notebook is to demonstrate detecting the various vulnerabilities using joern's CPGQL queries.

We begin by setting up the basics for joern.

In [1]:
import asyncio

from joern_lib import client, workspace
from joern_lib.detectors import common as cpg, java
from joern_lib.utils import print_table, print_tree, print_flows

joern_host = "http://joern:9000"
cpggen_host = "http://cpggen:7072"
joern_username = "admin"
joern_password  = "admin"

## Generate CPG using cpggen

To improve performance, CPG for this project is generated using the cpggen server and then imported onto the joern server. The default docker-compose starts a cpggen server with the hostname `cpggen` and port `7072`.

We use the `create_cpg` api from workspace to create a CPG from the git repo directly. This single call would handle both generation of the cpg and import onto joern.

In [3]:
async def generate_cpg():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await workspace.create_cpg(connection, "https://github.com/HooliCorp/vuln-spring")
    print (res)
asyncio.run(generate_cpg())

╭───────────────────────────────── CPGQL Query ─────────────────────────────────╮
│ os.exists(os.Path("/tmp/cpggen_cpg_outg4kolij1/cpggenu421pp76-java.cpg.bin")) │
╰───────────────────────────────────────────────────────────────────────────────╯
╭───────────────────────────────────────── CPGQL Query ──────────────────────────────────────────╮
│ importCpg("/tmp/cpggen_cpg_outg4kolij1/cpggenu421pp76-java.cpg.bin", "cpggenu421pp76-javasrc") │
╰────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ CPGQL Query ─╮
│ save          │
╰───────────────╯
{'app': 'cpggenu421pp76-javasrc', 'cpg': '/tmp/cpggen_cpg_outg4kolij1/cpggenu421pp76-java.cpg.bin', 'cpg_frontend_invocation': '/usr/local/bin/javasrc2cpg -J-Xmx107G -o /tmp/cpggen_cpg_outg4kolij1/cpggenu421pp76-java.cpg.bin /tmp/cpggenu421pp76', 'group': 'cpggenu421pp76', 'language': 'javasrc', 'sbom': '/tmp/cpggen_cpg_outg4kolij1/cpggenu421pp76-java.bom.xml', 'sbom_invocation': 'cdxgen -r -t java -

## Inspect methods

With a fresh CPG imported, one of the initial activities to perform as a threat hunter is to inspect the methods in the application. Visually inspecting the methods, would help understand not just the custom code but also the frameworks and libraries used.

joern-lib offers a convenient api called `list_methods` to simplify the inspection. You can invoke this method without arguments to receive a full list of methods or provide a regex to narrow the results.

In [5]:
async def inspect_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_methods(connection)
    print_table(res, title="vuln-spring methods")
asyncio.run(inspect_methods())

╭─────────────────────────── CPGQL Query ───────────────────────────╮
│ cpg.method.whereNot(_.name(".*<(operator|init)>.*")).toJsonPretty │
╰───────────────────────────────────────────────────────────────────╯
                                                                                vuln-spring methods                                                                                 
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                 ┃ fullName            ┃ signature            ┃ lineNumberEnd ┃ _label ┃ code                ┃ isExternal ┃ lineNumber ┃   id ┃ order ┃ filename             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ doFilter             │ com.example.vulnspr │ <unresolvedSignatur

In [6]:
async def search_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_methods(connection, ".*vulnspring.*")
    print_table(res, title="Methods Search Result")
asyncio.run(search_methods())

╭──────────────────────────────────────── CPGQL Query ─────────────────────────────────────────╮
│ cpg.method.fullName(".*vulnspring.*").whereNot(_.name(".*<(operator|init)>.*")).toJsonPretty │
╰──────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                               Methods Search Result                                                                                
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name              ┃ fullName             ┃ signature            ┃ lineNumberEnd ┃ _label ┃ code                 ┃ isExternal ┃ lineNumber ┃   id ┃ order ┃ filename              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━━

## Inspect annotations

Inspecting annotations is a quicker way to identify the various http routes, components, and controllers in web applications. You can use the `list_annotations` api from cpg.

In [8]:
async def inspect_annotations():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_annotations(connection)
    print_table(res, title="vuln-spring annotations")
asyncio.run(inspect_annotations())

╭──────── CPGQL Query ────────╮
│ cpg.annotation.toJsonPretty │
╰─────────────────────────────╯
                                                                              vuln-spring annotations                                                                               
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┓
┃ name                  ┃ code                                           ┃ order ┃ _label     ┃ argumentIndex ┃ fullName                                       ┃ lineNumber ┃   id ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━┩
│ Override              │ @Override                                      │     8 │ ANNOTATION │            -1 │ <unresolvedNamespace>.Override                 │         23 │   92 │

## Identify complex methods

Complex methods typically contain a number of control structures. A convenient function called `get_complex_functions` could be used with the `n` argument to identify such methods of interest.

In [10]:
async def inspect_complex_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.get_complex_methods(connection, n=3)
    print_table(res, title="vuln-spring complex methods")
asyncio.run(inspect_complex_methods())

╭───────────────────────────────────────────── CPGQL Query ─────────────────────────────────────────────╮
│ ({cpg.method.internal.filter(_.controlStructure.size > 3).nameNot("<global>")}).location.toJsonPretty │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                            vuln-spring complex methods                                                                             
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ packageName      ┃ symbol       ┃ filename         ┃ className       ┃ methodFullName   ┃ lineNumber ┃ classShortName ┃ methodShortName ┃ node            ┃ _label   ┃ nodeLabel ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇

## Get the call tree

The next step in our journey is to visualize the call tree for some interesting methods to dig deeper.

In [12]:
async def get_call_tree():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.get_call_tree(connection, "com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)")
    print_tree(res)
asyncio.run(get_call_tree())

╭─────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────╮
│                                                                                                           │
│ import scala.collection.mutable.ListBuffer                                                                │
│ def printDashes(count: Int) = {                                                                           │
│     var tabStr = "+--- "                                                                                  │
│     var i = 0                                                                                             │
│     while (i < count) {                                                                                   │
│         tabStr = "|    " + tabStr                                                                         │
│         i += 1                                                                                            │
│     }   

## List HTTP routes and filters

joern-lib comes packaged with useful api methods to simplify code analysis. `list_http_routes` and `list_http_filters` are starter methods to list some HTTP routes and filters (sources) for the application.

In [14]:
async def get_routes():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await java.list_http_routes(connection)
    print_table(res, title="vuln-spring http routes")
asyncio.run(get_routes())

╭──────────────────────────────────────────────────────────────────── CPGQL Query ────────────────────────────────────────────────────────────────────╮
│ cpg.method.internal.where(_.annotation.fullName("org\\.springframework\\.web\\.bind\\.annotation\\..*")).map(m => (m, m.annotation.l)).toJsonPretty │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                              vuln-spring http routes                                                                               
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ name             ┃ fullName         ┃ signature        ┃ lineNumberEnd ┃ _label ┃ code             ┃ isExternal ┃ lineNumber ┃   id ┃ order ┃ filename        ┃ annotation       ┃
┡

In [15]:
async def get_filters():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await java.list_http_filters(connection)
    print_table(res, title="vuln-spring web filters")
asyncio.run(get_filters())

╭────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────╮
│ cpg.typeDecl.where(_.annotation.fullName("javax\\.servlet\\.annotation\\.WebFilter")).map(m => (m, m.annotation.l)).toJsonPretty │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                              vuln-spring web filters                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ inheritsFromTypeFul ┃               ┃                      ┃           ┃                     ┃            ┃            ┃    ┃       ┃                      ┃                     ┃
┃ lName               ┃ name          ┃ fullName          

# Automated Analysis

Java detector has a useful method to suggest some flows based on simple logic to identify sources and sinks. The results might include false positives since not every source or sink method might be valid from a vulnerability perspective.

In [17]:
async def show_some_flows():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await java.suggest_flows(connection)
asyncio.run(show_some_flows())

╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│ def source =                                                                                                                                                                     │
│         (cpg.method.internal.where(_.annotation.fullName("(?i).*(cloud|framework|data|http|net|socket|io|security|text|xml|json|proto|rpc|java).*")).whereNot(_.fullName(".*<unr │
│                                                                                                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query

# OWASP Top 10

Having mastered the basics of Joern and joern-lib, it is time to dive into detecting OWASP top 10 vulnerabilities in this project. To make life easy, we first tag all spring request parameters as `attacker-controlled`. This would help simplify future data-flow queries.

In [19]:
async def create_attacker_controlled_tag():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await cpg.create_tags(connection, query='cpg.annotation.fullName("org.springframework.web.bind.annotation.Request(Param|Body)").parameter.typeFullNameExact("java.lang.String")', tags=["attacker-controlled"])
    res = await cpg.list_tags(connection, name="attacker-controlled", is_parameter=True)
    print_table(res)
asyncio.run(create_attacker_controlled_tag())

╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                  │
│                 cpg.annotation.fullName("org.springframework.web.bind.annotation.Request(Param|Body)").parameter.typeFullNameExact("java.lang.String").newTagNode("attacker-cont │
│                 run.commit                                                                                                                                                       │
│                 save                                                                                                                                                             │
│                                                                                              

## A01:2021 – Broken Access Control

Directory Traversal vulnerability exists when an attacker-controlled parameter is passed to a file read operation.

In [21]:
async def detect_traversal():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullNameExact("java.io.File.<init>:void(java.lang.String)").parameter.order(1)')
asyncio.run(detect_traversal())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭──────────────────────────────────────────── CPGQL Query ─────────────────────────────────────────────╮
│ def sink = cpg.method.fullNameExact("java.io.File.<init>:void(java.lang.String)").parameter.order(1) │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_axjwytvi.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ dbpath                │
│ sanitizedValue        │
╰───────────────────────╯
╭─────────────────────────────────────

Sensitive data exposure arises when a sensitive data such as password or PII data is exposed to an external service or logging method without obfuscation.

In [23]:
async def detect_sde():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    # Tag certain parameters as sensitive
    await cpg.create_tags(connection, query='cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.name("(?i).*(password|token|key|secret).*")', tags=["sensitive"])
    # Query based on the tag
    await client.df(connection, {"parameter": "sensitive"}, 'cpg.method.fullName("org.slf4j.Logger.*").parameter.orderGt(0)')
asyncio.run(detect_sde())

╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                  │
│                 cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.name("(?i).*(password|token|key|secret).*").newTagNode("sensitive │
│                 run.commit                                                                                                                                                       │
│                 save                                                                                                                                                             │
│                                                                                              

JWT encode/decode misconfigurations

In [25]:
async def detect_jwt_issues():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("com.auth0.jwt.*").parameter')
    print("*** WARNING: JWT none algorithm used here ***\n")
    # Find usages of JWT none algorithm
    await client.df(connection, 'cpg.method.fullNameExact("com.auth0.jwt.algorithms.Algorithm.none:com.auth0.jwt.algorithms.Algorithm()").methodReturn', 'cpg.method.fullName("com.auth0.jwt.*").parameter.order(1)')
asyncio.run(detect_jwt_issues())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭──────────────────────── CPGQL Query ────────────────────────╮
│ def sink = cpg.method.fullName("com.auth0.jwt.*").parameter │
╰─────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_j_mk7un0.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ jwtString             │
╰───────────────────────╯
╭─────────────────────────────────────────── Data Flow ────────────────────────────────────────────╮
│ src/main/java/com/example/vulnspring/WebController.java:287 jwt() @RequestParam Stri

Open redirect

In [27]:
async def detect_open_redirect():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.call(Operators.addition).code(".*redirect.*").method.methodReturn')
asyncio.run(detect_open_redirect())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭────────────────────────────────── CPGQL Query ───────────────────────────────────╮
│ def sink = cpg.call(Operators.addition).code(".*redirect.*").method.methodReturn │
╰──────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_oo5ba8gh.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ desk                  │
│ helpDeskLocation      │
╰───────────────────────╯
╭──────────────────────────────────────────────────── Data Flow ──────────────────────────────────

## A03:2021 – Injection

### SQL Injection

The first style of SQL Injection we will detect with joern is an end-to-end example where attacker-controlled string parameter is used directly in a query without parameterization.

It is important to limit only to string parameter since numeric values cannot result in a successful injection. We first show the implementation using the `RequestParam` parameter.

In [29]:
async def detect_sqli():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String")',
        'cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.index(0)')
asyncio.run(detect_sqli())

╭─────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────╮
│ def source = cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String") │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────╮
│ def sink = cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.index(0) │
╰───────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_83tfdc10.json" │
╰───────

The source parameter in the above example could be rewritten to use the custom `attacker-controlled` tag that we created. This style is adopted in the rest of the exercise.

In [31]:
async def detect_sqli_attacker_controlled():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.order(1)')
asyncio.run(detect_sqli_attacker_controlled())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────╮
│ def sink = cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.order(1) │
╰───────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_mfa72j3r.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ password              │
│ query                 │
╰───────────────────────╯
╭──────────────────────────────────────────────

The sink query could be rewritten to use a tag called `sql-sink`

In [33]:
async def create_sql_sink_tag():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await cpg.create_tags(connection, query='cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.order(1)', tags=["sql-sink"])
    res = await cpg.list_tags(connection, name="sql-sink", is_parameter=True)
    print_table(res)
asyncio.run(create_sql_sink_tag())

╭──────────────────────────────────────────────────────────── CPGQL Query ────────────────────────────────────────────────────────────╮
│                                                                                                                                     │
│                 cpg.method.fullName("org.springframework.jdbc.core.JdbcTemplate.*").parameter.order(1).newTagNode("sql-sink").store │
│                 run.commit                                                                                                          │
│                 save                                                                                                                │
│                                                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭────────────────── CPGQL Query ────────────────

In [34]:
async def detect_sqli_taggged():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, {"parameter": "sql-sink"})
asyncio.run(detect_sqli_taggged())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭───────────────── CPGQL Query ─────────────────╮
│ def sink = cpg.tag.name("sql-sink").parameter │
╰───────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_0d5j48p9.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ password              │
│ query                 │
╰───────────────────────╯
╭────────────────────────────────────────────────────────────────────────────── Data Flow ──────────────────────────────────────────────────────────────────────────────╮
│ src/main/java/com/example/vulns

In [35]:
async def detect_reflected_xss():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String")', 'cpg.method.fullName("org.springframework.ui.Model.*").parameter.order(2)', filter=['skip_checks'])
asyncio.run(detect_reflected_xss())

╭─────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────╮
│ def source = cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String") │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭──────────────────────────────────── CPGQL Query ────────────────────────────────────╮
│ def sink = cpg.method.fullName("org.springframework.ui.Model.*").parameter.order(2) │
╰─────────────────────────────────────────────────────────────────────────────────────╯
╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).filter(m => m.elements.code(".*(check|valid|sanit|escape|clean|safe|seria

## A05:2021 – Security Misconfiguration

XXE

In [37]:
async def detect_xxe():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("javax.xml.parsers.DocumentBuilder.*").parameter.order(1)')
asyncio.run(detect_xxe())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭────────────────────────────────────── CPGQL Query ───────────────────────────────────────╮
│ def sink = cpg.method.fullName("javax.xml.parsers.DocumentBuilder.*").parameter.order(1) │
╰──────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_icjsqyzw.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ body                  │
╰───────────────────────╯
╭───────────────────────────────────────── Data Flow ──────────────────────────────────────────╮
│ s

## A07:2021 – Identification and Authentication Failures

Session Injection

In [39]:
async def detect_session_injection():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("javax.servlet.http.HttpSession.setAttribute.*").parameter.order(2)')
asyncio.run(detect_session_injection())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────── CPGQL Query ────────────────────────────────────────────╮
│ def sink = cpg.method.fullName("javax.servlet.http.HttpSession.setAttribute.*").parameter.order(2) │
╰────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_cuydorcd.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ username              │
╰───────────────────────╯
╭───────────────────────────────────────────────────────────── Data Fl

## A08:2021 – Software and Data Integrity Failures

Deserialization


In [41]:
async def detect_deserialization():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("java.io.ObjectInputStream.readObject.*").parameter.order(0)')
asyncio.run(detect_deserialization())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭──────────────────────────────────────── CPGQL Query ────────────────────────────────────────╮
│ def sink = cpg.method.fullName("java.io.ObjectInputStream.readObject.*").parameter.order(0) │
╰─────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_z3ejkxm_.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ encodedString         │
│ data                  │
│ ois                   │
╰───────────────────────╯
╭──────────────────────────────────────

## A09:2021 – Security Logging and Monitoring Failures

Log Forging

In [43]:
async def detect_log_forging():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.external.fullName("org.slf4j.Logger.*").parameter.orderGt(0)', filter=['skip_checks'])
asyncio.run(detect_log_forging())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭─────────────────────────────────── CPGQL Query ────────────────────────────────────╮
│ def sink = cpg.method.external.fullName("org.slf4j.Logger.*").parameter.orderGt(0) │
╰────────────────────────────────────────────────────────────────────────────────────╯
╭────────────────────────────────────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).filter(m => m.elements.code(".*(check|valid|sanit|escape|clean|safe|serialize|convert|authenticate|authorize|encode|encrypt).*").size == 0).map(m  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tain

## A10:2021 – Server-Side Request Forgery (SSRF)

In [45]:
async def detect_ssrf():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.external.fullName("java.util.Scanner.*").parameter.order(1)')
asyncio.run(detect_ssrf())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ def source = cpg.tag.name("attacker-controlled").parameter │
╰────────────────────────────────────────────────────────────╯
╭─────────────────────────────────── CPGQL Query ───────────────────────────────────╮
│ def sink = cpg.method.external.fullName("java.util.Scanner.*").parameter.order(1) │
╰───────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJson |> "/tmp/reachable_flows_9cumb2ze.json" │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ Tainted Identifiers ─╮
│ dbpath                │
│ sanitizedValue        │
│ remoteUrl             │
╰───────────────────────╯
╭──────────────────────────────────────────────────── Data Flow ─────